In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../../')
from noise2seg.models import Noise2Seg, Noise2SegConfig
import numpy as np
from csbdeep.utils import plot_history
from noise2seg.utils.misc_utils import combine_train_test_data, shuffle_train_data, augment_data
from noise2seg.utils.seg_utils import *
from noise2seg.utils.compute_precision_threshold import measure_precision
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from scipy import ndimage
import tensorflow as tf
import keras.backend as K
import urllib
import os
import zipfile
import pandas as pd

Using TensorFlow backend.


#### Read data and load model

In [2]:
def read_data(exp, noise_level):
    
    data_path = '/home/prakash/Desktop/fileserver_Noise2Seg/data/'
    gt_data = np.load(data_path+exp+'_n0'+'/test/test_data.npz', allow_pickle=True)
    test_data = np.load(data_path+exp+'_'+noise_level+'/test/test_data.npz', allow_pickle=True)
    gt_data = gt_data['X_test']
    test_data = test_data['X_test']
    return gt_data, test_data

In [3]:
def load_trained_model(exp, noise_level, run, fraction):
    
    exp_path = '/home/prakash/Desktop/fileserver_Noise2Seg/experiments/'
    base_dir = exp_path+exp+'_'+noise_level+'_run'+str(run)+'/fraction_'+fraction+'/'
    model_name = exp+'_'+noise_level+'_run'+str(run)+'_model'
    n2s_model = Noise2Seg(None, model_name, base_dir)
    return n2s_model

#### PSNR computation

In [4]:
def PSNR(gt, img):
    mse = np.mean(np.square(gt - img))
    return 20 * np.log10(np.max(gt)-np.min(gt)) - 10 * np.log10(mse)

In [5]:
def compute_mean_psnr(exp, noise_level, fraction, gt_data, test_data):
    psnr_per_run = []
    sem_ = []
    runs = np.arange(1,6)
    for run_idx in runs:
        n2s_model = load_trained_model(exp, noise_level, run_idx, fraction)
        denoised_images = []
        for i in range(test_data.shape[0]):
            denoised_ = n2s_model.predict(test_data[i].astype(np.float32),'YX')[...,0]
            denoised_images.append(denoised_)
        denoised_images = np.array(denoised_images)

        psnrs = []
        for gt, img in zip(gt_data, denoised_images):
            psnrs.append(PSNR(gt, img))

        psnrs = np.array(psnrs)
        psnr_per_run.append(np.round(np.mean(psnrs), 2))
    return np.mean(psnr_per_run), np.std(psnr_per_run)/np.sqrt(len(runs))

#### Compute results for DSB

In [6]:
gt_data_n20, test_data_n20 = read_data('DSB2018', 'n20')
gt_data_n10, test_data_n10 = read_data('DSB2018', 'n10')

n2v_dsb_n10, sem_n2v_dsb_n10 = compute_mean_psnr(exp='finDenoise_dsb', noise_level='n10', fraction='100.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)
n2v_dsb_n20, sem_n2v_dsb_n20 = compute_mean_psnr(exp='finDenoise_dsb', noise_level='n20', fraction='100.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)


denoiseg_dsb_n20_fraction_025, sem_dsb_n20_fraction_025 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n20', fraction='0.25', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_dsb_n10_fraction_025, sem_dsb_n10_fraction_025 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n10', fraction='0.25', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_dsb_n20_fraction_05, sem_dsb_n20_fraction_05 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n20', fraction='0.5', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_dsb_n10_fraction_05, sem_dsb_n10_fraction_05 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n10', fraction='0.5', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_dsb_n20_fraction_1, sem_dsb_n20_fraction_1 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n20', fraction='1.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_dsb_n10_fraction_1, sem_dsb_n10_fraction_1 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n10', fraction='1.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_dsb_n20_fraction_2, sem_dsb_n20_fraction_2 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n20', fraction='2.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_dsb_n10_fraction_2, sem_dsb_n10_fraction_2 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n10', fraction='2.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_dsb_n20_fraction_4, sem_dsb_n20_fraction_4 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n20', fraction='4.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_dsb_n10_fraction_4, sem_dsb_n10_fraction_4 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n10', fraction='4.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_dsb_n20_fraction_8, sem_dsb_n20_fraction_8 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n20', fraction='8.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_dsb_n10_fraction_8, sem_dsb_n10_fraction_8 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n10', fraction='8.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_dsb_n20_fraction_16, sem_dsb_n20_fraction_16 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n20', fraction='16.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_dsb_n10_fraction_16, sem_dsb_n10_fraction_16 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n10', fraction='16.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_dsb_n20_fraction_32, sem_dsb_n20_fraction_32 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n20', fraction='32.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_dsb_n10_fraction_32, sem_dsb_n10_fraction_32 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n10', fraction='32.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_dsb_n20_fraction_64, sem_dsb_n20_fraction_64 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n20', fraction='64.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_dsb_n10_fraction_64, sem_dsb_n10_fraction_64 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n10', fraction='64.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_dsb_n20_fraction_100, sem_dsb_n20_fraction_100 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n20', fraction='100.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_dsb_n10_fraction_100, sem_dsb_n10_fraction_100 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n10', fraction='100.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_be

In [67]:
n2v_dsb_n10_ = str(n2v_dsb_n10)[:5]+'±'+str(sem_n2v_dsb_n10)[:5]
n2v_dsb_n20_ = str(n2v_dsb_n20)[:5]+'±'+str(sem_n2v_dsb_n20)[:5]
denoiseg_dsb_n20_fraction_025_ = str(denoiseg_dsb_n20_fraction_025)[:5] +'±'+str(sem_dsb_n20_fraction_025)[:5]
denoiseg_dsb_n10_fraction_025_ = str(denoiseg_dsb_n10_fraction_025)[:5] +'±'+str(sem_dsb_n10_fraction_025)[:5]
denoiseg_dsb_n20_fraction_05_ = str(denoiseg_dsb_n20_fraction_05)[:5] +'±'+str(sem_dsb_n20_fraction_05)[:5]
denoiseg_dsb_n10_fraction_05_ = str(denoiseg_dsb_n10_fraction_05)[:5] +'±'+str(sem_dsb_n10_fraction_05)[:5]
denoiseg_dsb_n20_fraction_1_ = str(denoiseg_dsb_n20_fraction_1)[:5] +'±'+str(sem_dsb_n20_fraction_1)[:5]
denoiseg_dsb_n10_fraction_1_ = str(denoiseg_dsb_n10_fraction_1)[:5] +'±'+str(sem_dsb_n10_fraction_1)[:5]
denoiseg_dsb_n20_fraction_2_ = str(denoiseg_dsb_n20_fraction_2)[:5] +'±'+str(sem_dsb_n20_fraction_2)[:5]
denoiseg_dsb_n10_fraction_2_ = str(denoiseg_dsb_n10_fraction_2)[:5] +'±'+str(sem_dsb_n10_fraction_2)[:5]
denoiseg_dsb_n20_fraction_4_ = str(denoiseg_dsb_n20_fraction_4)[:5] +'±'+str(sem_dsb_n20_fraction_4)[:5]
denoiseg_dsb_n10_fraction_4_ = str(denoiseg_dsb_n10_fraction_4)[:5] +'±'+str(sem_dsb_n10_fraction_4)[:5]
denoiseg_dsb_n20_fraction_8_ = str(denoiseg_dsb_n20_fraction_8)[:5] +'±'+str(sem_dsb_n20_fraction_8)[:5]
denoiseg_dsb_n10_fraction_8_ = str(denoiseg_dsb_n10_fraction_8)[:5] +'±'+str(sem_dsb_n10_fraction_8)[:5]
denoiseg_dsb_n20_fraction_16_ = str(denoiseg_dsb_n20_fraction_16)[:5] +'±'+str(sem_dsb_n20_fraction_16)[:5]
denoiseg_dsb_n10_fraction_16_ = str(denoiseg_dsb_n10_fraction_16)[:5] +'±'+str(sem_dsb_n10_fraction_16)[:5]
denoiseg_dsb_n20_fraction_32_ = str(denoiseg_dsb_n20_fraction_32)[:5] +'±'+str(sem_dsb_n20_fraction_32)[:5]
denoiseg_dsb_n10_fraction_32_ = str(denoiseg_dsb_n10_fraction_32)[:5] +'±'+str(sem_dsb_n10_fraction_32)[:5]
denoiseg_dsb_n20_fraction_64_ = str(denoiseg_dsb_n20_fraction_64)[:5] +'±'+str(sem_dsb_n20_fraction_64)[:5]
denoiseg_dsb_n10_fraction_64_ = str(denoiseg_dsb_n10_fraction_64)[:5] +'±'+str(sem_dsb_n10_fraction_64)[:5]
denoiseg_dsb_n20_fraction_100_ = str(denoiseg_dsb_n20_fraction_100)[:5] +'±'+str(sem_dsb_n20_fraction_100)[:5]
denoiseg_dsb_n10_fraction_100_ = str(denoiseg_dsb_n10_fraction_100)[:5] +'±'+str(sem_dsb_n10_fraction_100)[:5]

In [68]:
styles = [dict(selector="caption", 
    props=[("text-align", "center"),
    ("font-size", "120%"),
    ("color", 'black')])] 

In [69]:
heads = ['DenoiSeg (GT=10)', 'DenoiSeg (GT=19)', 'DenoiSeg (GT=38)', 'DenoiSeg (GT=76)', 'DenoiSeg (GT=152)', 'DenoiSeg (GT=304)', 'DenoiSeg (GT=608)', 'DenoiSeg (GT=1216)', 'DenoiSeg (GT=2432)', 'DenoiSeg (GT=3800)', 'Noise2Void']
noise_levels = ['n10', 'n20']
n10_scores = [denoiseg_dsb_n10_fraction_025_, denoiseg_dsb_n10_fraction_05_, denoiseg_dsb_n10_fraction_1_, denoiseg_dsb_n10_fraction_2_, denoiseg_dsb_n10_fraction_4_, denoiseg_dsb_n10_fraction_8_, denoiseg_dsb_n10_fraction_16_, denoiseg_dsb_n10_fraction_32_, denoiseg_dsb_n10_fraction_64_, denoiseg_dsb_n10_fraction_100_, n2v_dsb_n10_]
n20_scores = [denoiseg_dsb_n20_fraction_025_, denoiseg_dsb_n20_fraction_05_, denoiseg_dsb_n20_fraction_1_, denoiseg_dsb_n20_fraction_2_, denoiseg_dsb_n20_fraction_4_, denoiseg_dsb_n20_fraction_8_, denoiseg_dsb_n20_fraction_16_, denoiseg_dsb_n20_fraction_32_, denoiseg_dsb_n20_fraction_64_, denoiseg_dsb_n20_fraction_100_, n2v_dsb_n20_]
scores = np.array([n10_scores, n20_scores])
df = pd.DataFrame(scores,index=noise_levels,columns=heads).round(decimals=3)

print('Mean PSNR for DSB in dB. DenoiSeg setup is with alpha = 0.5')
df.style.set_caption('Mean PSNR for DSB in dB. DenoiSeg setup is with alpha = 0.5').set_table_styles(styles)

Mean PSNR in dB. DenoiSeg setup is with alpha = 0.5


,DenoiSeg (GT=10),DenoiSeg (GT=19),DenoiSeg (GT=38),DenoiSeg (GT=76),DenoiSeg (GT=152),DenoiSeg (GT=304),DenoiSeg (GT=608),DenoiSeg (GT=1216),DenoiSeg (GT=2432),DenoiSeg (GT=3800),Noise2Void
n10,37.57±0.070,37.64±0.111,37.61±0.110,37.51±0.109,37.54±0.107,37.62±0.025,37.83±0.048,37.86±0.040,37.67±0.111,37.81±0.049,38.01±0.0590
n20,35.38±0.083,35.22±0.184,35.04±0.089,34.92±0.147,35.04±0.120,35.27±0.088,34.90±0.109,35.27±0.026,35.24±0.084,35.19±0.079,35.53±0.0267


#### Compute results for Fly Wing

In [6]:
gt_data_n20, test_data_n20 = read_data('Flywing', 'n20')
gt_data_n10, test_data_n10 = read_data('Flywing', 'n10')
n2v_flywing_n10, sem_n2v_flywing_n10 = compute_mean_psnr(exp='finDenoise_flywing', noise_level='n10', fraction='100.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)
n2v_flywing_n20, sem_n2v_flywing_n20 = compute_mean_psnr(exp='finDenoise_flywing', noise_level='n20', fraction='100.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)


denoiseg_flywing_n20_fraction_0125, sem_flywing_n20_fraction_0125 = compute_mean_psnr(exp='alpha0.5_flywing', noise_level='n20', fraction='0.125', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_flywing_n10_fraction_0125, sem_flywing_n10_fraction_0125 = compute_mean_psnr(exp='alpha0.5_flywing', noise_level='n10', fraction='0.125', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_flywing_n20_fraction_025, sem_flywing_n20_fraction_025 = compute_mean_psnr(exp='alpha0.5_flywing', noise_level='n20', fraction='0.25', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_flywing_n10_fraction_025, sem_flywing_n10_fraction_025 = compute_mean_psnr(exp='alpha0.5_flywing', noise_level='n10', fraction='0.25', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_flywing_n20_fraction_05, sem_flywing_n20_fraction_05 = compute_mean_psnr(exp='alpha0.5_flywing', noise_level='n20', fraction='0.5', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_flywing_n10_fraction_05, sem_flywing_n10_fraction_05 = compute_mean_psnr(exp='alpha0.5_flywing', noise_level='n10', fraction='0.5', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_flywing_n20_fraction_1, sem_flywing_n20_fraction_1 = compute_mean_psnr(exp='alpha0.5_flywing', noise_level='n20', fraction='1.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_flywing_n10_fraction_1, sem_flywing_n10_fraction_1 = compute_mean_psnr(exp='alpha0.5_flywing', noise_level='n10', fraction='1.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_flywing_n20_fraction_2, sem_flywing_n20_fraction_2 = compute_mean_psnr(exp='alpha0.5_flywing', noise_level='n20', fraction='2.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_flywing_n10_fraction_2, sem_flywing_n10_fraction_2 = compute_mean_psnr(exp='alpha0.5_flywing', noise_level='n10', fraction='2.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)


Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_be

In [7]:
n2v_flywing_n10_ = str(n2v_flywing_n10)[:5]+'±'+str(sem_n2v_flywing_n10)[:5]
n2v_flywing_n20_ = str(n2v_flywing_n20)[:5]+'±'+str(sem_n2v_flywing_n20)[:5]
denoiseg_flywing_n20_fraction_0125_ = str(denoiseg_flywing_n20_fraction_0125)[:5] +'±'+str(sem_flywing_n20_fraction_0125)[:5]
denoiseg_flywing_n10_fraction_0125_ = str(denoiseg_flywing_n10_fraction_0125)[:5] +'±'+str(sem_flywing_n10_fraction_0125)[:5]
denoiseg_flywing_n20_fraction_025_ = str(denoiseg_flywing_n20_fraction_025)[:5] +'±'+str(sem_flywing_n20_fraction_025)[:5]
denoiseg_flywing_n10_fraction_025_ = str(denoiseg_flywing_n10_fraction_025)[:5] +'±'+str(sem_flywing_n10_fraction_025)[:5]
denoiseg_flywing_n20_fraction_05_ = str(denoiseg_flywing_n20_fraction_05)[:5] +'±'+str(sem_flywing_n20_fraction_05)[:5]
denoiseg_flywing_n10_fraction_05_ = str(denoiseg_flywing_n10_fraction_05)[:5] +'±'+str(sem_flywing_n10_fraction_05)[:5]
denoiseg_flywing_n20_fraction_1_ = str(denoiseg_flywing_n20_fraction_1)[:5] +'±'+str(sem_flywing_n20_fraction_1)[:5]
denoiseg_flywing_n10_fraction_1_ = str(denoiseg_flywing_n10_fraction_1)[:5] +'±'+str(sem_flywing_n10_fraction_1)[:5]
denoiseg_flywing_n20_fraction_2_ = str(denoiseg_flywing_n20_fraction_2)[:5] +'±'+str(sem_flywing_n20_fraction_2)[:5]
denoiseg_flywing_n10_fraction_2_ = str(denoiseg_flywing_n10_fraction_2)[:5] +'±'+str(sem_flywing_n10_fraction_2)[:5]

In [9]:
styles = [dict(selector="caption", 
    props=[("text-align", "center"),
    ("font-size", "120%"),
    ("color", 'black')])] 

In [13]:
heads = ['DenoiSeg (GT=2)', 'DenoiSeg (GT=4)', 'DenoiSeg (GT=7)', 'DenoiSeg (GT=11)', 'DenoiSeg (GT=29)', 'Noise2Void']
noise_levels = ['n10', 'n20']
n10_scores = [denoiseg_flywing_n10_fraction_0125_, denoiseg_flywing_n10_fraction_025_, denoiseg_flywing_n10_fraction_05_, denoiseg_flywing_n10_fraction_1_, denoiseg_flywing_n10_fraction_2_, n2v_flywing_n10_]
n20_scores = [denoiseg_flywing_n20_fraction_0125_, denoiseg_flywing_n20_fraction_025_, denoiseg_flywing_n20_fraction_05_, denoiseg_flywing_n20_fraction_1_, denoiseg_flywing_n20_fraction_2_, n2v_flywing_n20_]
scores = np.array([n10_scores, n20_scores])
df = pd.DataFrame(scores,index=noise_levels,columns=heads).round(decimals=3)

print('Mean PSNR for Fly Wing in dB. DenoiSeg setup is with alpha = 0.5')
df.style.set_caption('Mean PSNR for Fly Wing in dB. DenoiSeg setup is with alpha = 0.5').set_table_styles(styles)

Mean PSNR for Fly Wing in dB. DenoiSeg setup is with alpha = 0.5


,DenoiSeg (GT=2),DenoiSeg (GT=4),DenoiSeg (GT=7),DenoiSeg (GT=11),DenoiSeg (GT=29),Noise2Void
n10,33.12±0.015,33.09±0.029,33.04±0.041,33.05±0.032,32.93±0.095,33.16±0.0121
n20,30.45±0.205,30.64±0.034,30.61±0.036,30.63±0.014,30.60±0.032,30.72±0.0183


#### Compute results for Mouse Nuclei

In [6]:
gt_data_n20, test_data_n20 = read_data('Mouse', 'n20')
gt_data_n10, test_data_n10 = read_data('Mouse', 'n10')
n2v_mouse_n10, sem_n2v_mouse_n10 = compute_mean_psnr(exp='finDenoise_nmouse', noise_level='n10', fraction='100.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)
n2v_mouse_n20, sem_n2v_mouse_n20 = compute_mean_psnr(exp='finDenoise_nmouse', noise_level='n20', fraction='100.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_mouse_n20_fraction_0125, sem_mouse_n20_fraction_0125 = compute_mean_psnr(exp='alpha0.5_nmouse', noise_level='n20', fraction='0.125', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_mouse_n10_fraction_0125, sem_mouse_n10_fraction_0125 = compute_mean_psnr(exp='alpha0.5_nmouse', noise_level='n10', fraction='0.125', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_mouse_n20_fraction_025, sem_mouse_n20_fraction_025 = compute_mean_psnr(exp='alpha0.5_nmouse', noise_level='n20', fraction='0.25', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_mouse_n10_fraction_025, sem_mouse_n10_fraction_025 = compute_mean_psnr(exp='alpha0.5_nmouse', noise_level='n10', fraction='0.25', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_mouse_n20_fraction_05, sem_mouse_n20_fraction_05 = compute_mean_psnr(exp='alpha0.5_nmouse', noise_level='n20', fraction='0.5', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_mouse_n10_fraction_05, sem_mouse_n10_fraction_05 = compute_mean_psnr(exp='alpha0.5_nmouse', noise_level='n10', fraction='0.5', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_mouse_n20_fraction_1, sem_mouse_n20_fraction_1 = compute_mean_psnr(exp='alpha0.5_nmouse', noise_level='n20', fraction='1.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_mouse_n10_fraction_1, sem_mouse_n10_fraction_1 = compute_mean_psnr(exp='alpha0.5_nmouse', noise_level='n10', fraction='1.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

denoiseg_mouse_n20_fraction_2, sem_mouse_n20_fraction_2 = compute_mean_psnr(exp='alpha0.5_nmouse', noise_level='n20', fraction='2.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
denoiseg_mouse_n10_fraction_2, sem_mouse_n10_fraction_2 = compute_mean_psnr(exp='alpha0.5_nmouse', noise_level='n10', fraction='2.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)

Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_be

In [10]:
n2v_mouse_n10_ = str(n2v_mouse_n10)[:5]+'±'+str(sem_n2v_mouse_n10)[:5]
n2v_mouse_n20_ = str(n2v_mouse_n20)[:5]+'±'+str(sem_n2v_mouse_n20)[:5]
denoiseg_mouse_n20_fraction_0125_ = str(denoiseg_mouse_n20_fraction_0125)[:5] +'±'+str(sem_mouse_n20_fraction_0125)[:5]
denoiseg_mouse_n10_fraction_0125_ = str(denoiseg_mouse_n10_fraction_0125)[:5] +'±'+str(sem_mouse_n10_fraction_0125)[:5]
denoiseg_mouse_n20_fraction_025_ = str(denoiseg_mouse_n20_fraction_025)[:5] +'±'+str(sem_mouse_n20_fraction_025)[:5]
denoiseg_mouse_n10_fraction_025_ = str(denoiseg_mouse_n10_fraction_025)[:5] +'±'+str(sem_mouse_n10_fraction_025)[:5]
denoiseg_mouse_n20_fraction_05_ = str(denoiseg_mouse_n20_fraction_05)[:5] +'±'+str(sem_mouse_n20_fraction_05)[:5]
denoiseg_mouse_n10_fraction_05_ = str(denoiseg_mouse_n10_fraction_05)[:5] +'±'+str(sem_mouse_n10_fraction_05)[:5]
denoiseg_mouse_n20_fraction_1_ = str(denoiseg_mouse_n20_fraction_1)[:5] +'±'+str(sem_mouse_n20_fraction_1)[:5]
denoiseg_mouse_n10_fraction_1_ = str(denoiseg_mouse_n10_fraction_1)[:5] +'±'+str(sem_mouse_n10_fraction_1)[:5]
denoiseg_mouse_n20_fraction_2_ = str(denoiseg_mouse_n20_fraction_2)[:5] +'±'+str(sem_mouse_n20_fraction_2)[:5]
denoiseg_mouse_n10_fraction_2_ = str(denoiseg_mouse_n10_fraction_2)[:5] +'±'+str(sem_mouse_n10_fraction_2)[:5]

In [11]:
styles = [dict(selector="caption", 
    props=[("text-align", "center"),
    ("font-size", "120%"),
    ("color", 'black')])] 

In [12]:
heads = ['DenoiSeg (GT=1)', 'DenoiSeg (GT=2)', 'DenoiSeg (GT=5)', 'DenoiSeg (GT=9)', 'DenoiSeg (GT=18)', 'Noise2Void']
noise_levels = ['n10', 'n20']
n10_scores = [denoiseg_mouse_n10_fraction_0125_, denoiseg_mouse_n10_fraction_025_, denoiseg_mouse_n10_fraction_05_, denoiseg_mouse_n10_fraction_1_, denoiseg_mouse_n10_fraction_2_, n2v_mouse_n10_]
n20_scores = [denoiseg_mouse_n20_fraction_0125_, denoiseg_mouse_n20_fraction_025_, denoiseg_mouse_n20_fraction_05_, denoiseg_mouse_n20_fraction_1_, denoiseg_mouse_n20_fraction_2_, n2v_mouse_n20_]
scores = np.array([n10_scores, n20_scores])
df = pd.DataFrame(scores,index=noise_levels,columns=heads).round(decimals=3)

print('Mean PSNR for Mouse Nuclei in dB. DenoiSeg setup is with alpha = 0.5')
df.style.set_caption('Mean PSNR for Mouse Nuclei in dB. DenoiSeg setup is with alpha = 0.5').set_table_styles(styles)

Mean PSNR for Mouse Nuclei in dB. DenoiSeg setup is with alpha = 0.5


,DenoiSeg (GT=1),DenoiSeg (GT=2),DenoiSeg (GT=5),DenoiSeg (GT=9),DenoiSeg (GT=18),Noise2Void
n10,37.42±0.104,37.11±0.323,37.46±0.040,37.27±0.093,37.02±0.157,37.86±0.015
n20,34.21±0.199,34.39±0.034,34.27±0.061,34.22±0.052,34.23±0.057,34.59±0.013
